<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto City - Part 2</font></h1>

## 1. Download required libraries

In [76]:
!conda install -c conda-forge beautifulsoup4 lxml html5lib requests geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4
    - geocoder
    - html5lib
    - lxml
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW 

## 2. Download html page and put in a soup object

In [70]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url, 'lxml')

## 3. Scrape html page and save data into an array

In [ ]:
# Using google location
import geocoder # import geocoder

table = soup.find('table',{'class':'wikitable sortable'})
listdict = []
for row in table.find_all('tr'):
    elements = row.find_all('td')
    if not elements:
        continue
    postalCode, borough, neighborhood = [element.text.strip() for element in elements]
    if borough != 'Not assigned':
        # initialize your variable to None
        lat_lng_coords = None
        
        # loop until you get the coordinates
        while(lat_lng_coords is None):
            g = geocoder.google('{}, Toronto, Ontario'.format(postalCode))
            lat_lng_coords = g.latlng

        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
        if neighborhood != 'Not assigned':
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':neighborhood, 'Latitude': latitude, 'Longitude': longitude})
        else:
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':borough, 'Latitude': latitude, 'Longitude': longitude})

In [117]:
# Using Geospatial_Coordinates.csv
table = soup.find('table',{'class':'wikitable sortable'})
listdict = []
coordinates = pd.read_csv('Geospatial_Coordinates.csv', delimiter = ',')
coordinates.columns = coordinates.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
for row in table.find_all('tr'):
    elements = row.find_all('td')
    if not elements:
        continue
    postalCode, borough, neighborhood = [element.text.strip() for element in elements]
    if borough != 'Not assigned':
        latitude = coordinates[coordinates.postal_code == postalCode].latitude.iloc[0]
        longitude = coordinates[coordinates.postal_code == postalCode].longitude.iloc[0]
        if neighborhood != 'Not assigned':
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':neighborhood, 'Latitude': latitude, 'Longitude': longitude})
        else:
            listdict.append({'PostalCode':postalCode, 'Borough':borough, 'Neighborhood':borough, 'Latitude': latitude, 'Longitude': longitude})

## 4. Create pandas dataframe from the array

In [118]:
# instantiate the dataframe
df = pd.DataFrame(listdict)

In [119]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
...,...,...,...,...,...
206,M8Z,Etobicoke,Kingsway Park South West,43.628841,-79.520999
207,M8Z,Etobicoke,Mimico NW,43.628841,-79.520999
208,M8Z,Etobicoke,The Queensway West,43.628841,-79.520999
209,M8Z,Etobicoke,Royal York South West,43.628841,-79.520999


In [120]:
df.shape

(211, 5)

In [121]:
df.to_csv('Toronto_Neighborhood.csv', index = False)